In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf


In [ ]:
train_data=pd.read_csv("../input/nlp-getting-started/train.csv")
train_data.head()

 In this Notebook I am going to Train some NLP Models in Tensorflow
 At the End I will also be making a Submission to the Competition Contradictory NLP Getting started with Disaster Tweets.
<!-- Also I will be using [MNLI](https://cims.nyu.edu/~sbowman/multinli/) for additional help with Inferences as I saw that in a Couple of Notebooks like one by [Andrej Marichenko](https://www.kaggle.com/andrej0marinchenko/public-score-0-95072-contradictory-my-dwatson) -->
Course Referred: https://www.udemy.com/course/tensorflow-developer-certificate-machine-learning-zero-to-mastery/




Initially I will be training some NLP Models and then Try and make a subsequent NLI model extrapolating the learnings.

In [ ]:
test_data=pd.read_csv("../input/nlp-getting-started/test.csv")
test_data.head()

In [ ]:
# Shuffling the Data for some Randomness and it is okay since it is not sequential
train_data_shuffle = train_data.sample(frac=1, random_state=42) # shuffle with random_state=42 for reproducibility
train_data_shuffle.head()

1 -> Disaster <br>
0-> Not Disaster

In [ ]:
# How many Disasters
train_data.target.value_counts()

In [ ]:
import random
random_index=random.randint(0,len(train_data)-5)
for row in train_data_shuffle[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

In [ ]:
from sklearn.model_selection import train_test_split

# Use train_test_split to split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_data_shuffle["text"].to_numpy(),
                                                                            train_data_shuffle["target"].to_numpy(),
                                                                            test_size=0.1, # dedicate 10% of samples to validation set
                                                                            random_state=42) # random state for reproducibility
test_sentences=test_data["text"].to_numpy()


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

Now we will be Embedding and Tokenization. <br>
This is an extremely crucial step while working with NLP Questions.<br>
Tokenization is Breaking down the sentencing in Numerical Formats and <br>
In Embedding Format each Word is turned into a vectorr and we can specify the size of the vector <br>
Representation comes in the form of a feature vector. <br>


In [ ]:
text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocabulary (all of the different words in your text)
                                    standardize="lower_and_strip_punctuation", # how to process text
                                    split="whitespace", # how to split tokens
                                    ngrams=None, # create groups of n-words?
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None) # how long should the output sequence of tokens be?
                                    # pad_to_max_tokens=True) # Not valid if using max_tokens=None

In [ ]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

In [ ]:
# Custom Vectorization with Custom Variable
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be 
text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [ ]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)
text_vectorizer.adapt(test_sentences)

In [ ]:
words_in_vocab = text_vectorizer.get_vocabulary()

In [ ]:
# Creating an Emmbedding
tf.random.set_seed(42)
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128, # set size of embedding vector
                             embeddings_initializer="uniform", # default, intialize randomly
                             input_length=max_length, # how long is each input
                             name="embedding_1")

In [ ]:
# So now Each statement in the training data Undergoes two Steps 
# 1 Vectorization
# 2 Embedding
# Now Building Multiple Models and them Ensembling them to get the Optimum Results
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline


In [ ]:
# Create tokenization and modelling pipeline
model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()), # convert words to numbers using tfidf
                    ("clf", MultinomialNB()) # model the text
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

In [ ]:
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

In [ ]:
baseline_preds = model_0.predict(val_sentences)

In [ ]:
# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
#   """
#   Calculates model accuracy, precision, recall and f1 score of a binary classification model.

#   Args:
#   -----
#   y_true = true labels in the form of a 1D array
#   y_pred = predicted labels in the form of a 1D array

#   Returns a dictionary of accuracy, precision, recall, f1-score.
#   """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

In [ ]:
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

In [ ]:
sub=pd.read_csv("../input/nlp-getting-started/sample_submission.csv")
sub.head()

In [ ]:
mod0_prediction=model_0.predict(test_sentences)

In [ ]:
mod0_prediction

In [ ]:
# The Predictions for Naive Bayes Model
# prediction=pd.DataFrame({'id':test_data.id,'target':mod0_prediction})
# prediction.to_csv('submission.csv',index=False)

In [ ]:
# prediction.head()

In [ ]:
import datetime

def create_tensorboard_callback(dir_name, experiment_name):
  """
  Creates a TensorBoard callback instand to store log files.
  Stores log files with the filepath:
    "dir_name/experiment_name/current_datetime/"
  Args:
    dir_name: target directory to store TensorBoard log files
    experiment_name: name of experiment directory (e.g. efficientnet_model_1)
  """
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=log_dir
  )
  print(f"Saving TensorBoard log files to: {log_dir}")
  return tensorboard_callback

<h2><b> Dense Model </b></h2>


In [ ]:
# Now we will be Building a TON  of Models and Testing Them to See which Ones perform the best and 
# at the end I will ensemble them or Stack them for Optimum performance
SAVE_DIR="model_logs"

In [ ]:
from tensorflow.keras import layers
inputs=layers.Input(shape=(1,),dtype=tf.string)
X=text_vectorizer(inputs)
# Creating an Embedding for the Input Layer
X=embedding(X)
# We need to perform this step so that we do not get predictions for each token
# But for each of the Tweets
X=layers.GlobalAveragePooling1D()(X)

# Dense Output Layer
# 1 Hidden Neuron as it is a Binary Classification
outputs=layers.Dense(1,activation="sigmoid")(X)
model_1=tf.keras.Model(inputs,outputs,name="Dense_Model")


In [ ]:
model_1.summary()

In [ ]:
model_1.compile(loss="binary_crossentropy",optimizer=tf.keras.optimizers.Adam(),metrics=["accuracy"])
model_1.history=model_1.fit(x=train_sentences,y=train_labels,epochs=10,validation_data
                           =(val_sentences,val_labels),
                           callbacks=create_tensorboard_callback(dir_name=SAVE_DIR,experiment_name="Model_1"))

In [ ]:
model_1.evaluate(val_sentences,val_labels)

In [ ]:
# Make predictions (these come back in the form of probabilities)
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs[:10] # only print out the first 10 prediction probabilities

In [ ]:
# Turn prediction probabilities into single-dimension tensor of floats
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs)) # squeeze removes single dimensions
model_1_preds[:20]

In [ ]:
model_1_results = calculate_results(y_true=val_labels, 
                                    y_pred=model_1_preds)
model_1_results

<h2> Visuallizing the Embedding <h2>

In [ ]:
# These are the Numerical Representation of each Token 
embed_weights=model_1.get_layer("embedding_1").get_weights()[0]
embed_weights

In [ ]:
embed_weights.shape

In [ ]:
# Projector Tool to visualize the Embedding Matrix
# # Code below is adapted from: https://www.tensorflow.org/tutorials/text/word_embeddings#retrieve_the_trained_word_embeddings_and_save_them_to_disk
import io

# # Create output writers
out_v = io.open("embedding_vectors.tsv", "w", encoding="utf-8")
out_m = io.open("embedding_metadata.tsv", "w", encoding="utf-8")

# # Write embedding vectors and words to file
for num, word in enumerate(words_in_vocab):
    if num == 0: 
#    continue # skip padding token
     vec = embed_weights[num]
     out_m.write(word + "\n") # write words to file
     out_v.write("\t".join([str(x) for x in vec]) + "\n") # write corresponding word vector to file
out_v.close()
out_m.close()




<a href="./embedding_metadata.tsv"> Download Metadata File </a> <br>
<a href="./embedding_vectors.tsv"> Download Vectors  File </a>

In [ ]:
# RNN LSTM and RGU Models
# Note: The reason we use a new embedding layer for each model is since the embedding layer is a learned representation of words (as numbers), if we were to use the same embedding layer (embedding_1) for each model, we'd be mixing what one model learned with the next. And because we want to compare our models later on, starting them with their own embedding layer each time is a better idea.
# LSTM Long Short Term Memory



Structure -->
Input-->
Tokenize-->
Embed-->
Layers of RNN-->
Output


In [ ]:
from tensorflow.keras import layers
inputs=layers.Input(shape=(1,),dtype=tf.string)
x=text_vectorizer(inputs)
x=embedding(x)
x=layers.LSTM(64,return_sequences=True)(x)
# x=layers.LSTM(64,return_sequences=True)(x)
# Now since it is an RNN output of one Layer will be fed to another 
x=layers.LSTM(64)(x)
x=layers.Dense(64,activation="relu")(x)
# The final Output Layer will be a Dense Layer and it will be 1 Hidden Neuron as Binary Labelling Output
outputs=layers.Dense(1,activation="sigmoid")(x)
model_2=tf.keras.Model(inputs,outputs,name="Model_2_LSTM")


In [ ]:
model_2.summary()
# While Sttacking RNN Models we need to return sequences mandatorily


In [ ]:
# Comiling and Fitting the Model
model_2.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(),metrics=["accuracy"])


In [ ]:
history_2_model=model_2.fit(x=train_sentences,y=train_labels, epochs=7,validation_data=(val_sentences,val_labels),
                           callbacks=[create_tensorboard_callback("SAVE_DIR","Model_2_LSTM")])

In [ ]:
model_2_predict_probs=model_2.predict(val_sentences)
model_2_predict_probs[:20]

In [ ]:
# Converting these Probabilites to Labels of 0 and 1
model_2_predictions=tf.squeeze(tf.round(model_2_predict_probs))
model_2_predictions

In [ ]:
model_2_res=calculate_results(y_true=val_labels,y_pred=model_2_predictions)
model_2_res

In [ ]:
# Building a GRU Powered RNN
 # Set random seed and create embedding layer (new embedding layer for each model)
tf.random.set_seed(42)
from tensorflow.keras import layers
model_3_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_3")

In [ ]:
inputs=layers.Input(shape=(1,),dtype=tf.string)
x=text_vectorizer(inputs)
x = model_3_embedding(x)
x = layers.GRU(64, return_sequences=True) (x)
x = layers.GRU(64, return_sequences=True)(x)
x = layers.GRU(64)(x) 
x = layers.Dense(64, activation="relu")(x) # optional dense layer after GRU cell
outputs = layers.Dense(1, activation="sigmoid")(x)
model_3 = tf.keras.Model(inputs, outputs, name="model_3_GRU")

In [ ]:
model_3.summary()

In [ ]:
model_3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=9,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "Model_3_GRU")])

In [ ]:
# Make predictions on the validation data
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs.shape, model_3_pred_probs[:10]

In [ ]:
# Convert prediction probabilities to prediction classes
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

In [ ]:
model_3_results = calculate_results(y_true=val_labels, 
                                    y_pred=model_3_preds)
model_3_results

<b><h2>Now on to a Bidirectional RNN Model

In [ ]:
tf.random.set_seed(42)
from tensorflow.keras import layers
model_4_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_4")

In [ ]:
inputs=layers.Input(shape=(1,),dtype=tf.string)
x=text_vectorizer(inputs)
x=model_4_embedding(x)
x=layers.Bidirectional(layers.LSTM(64,return_sequences=True))(x)
x=layers.Bidirectional(layers.LSTM(64))(x)
x=layers.Dense(64,activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_4=tf.keras.Model(inputs,outputs,name="Model_4_Bidirectional_RNN")



In [ ]:
model_4.summary()

In [ ]:
model_4.compile(loss="binary_crossentropy",optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
model_4_history = model_4.fit(train_sentences,
                              train_labels,
                              epochs=6,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "bidirectional_RNN")])

In [ ]:
# Make predictions with bidirectional RNN on the validation data
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

In [ ]:
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

In [ ]:
model_4_results = calculate_results(val_labels, model_4_preds)
model_4_results

<h2>TensorFlow Hub Pretrained Sentence Encoder<h2>

In [ ]:
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4") # load Universal Sentence Encoder

In [ ]:
# We can use this encoding layer in place of our text_vectorizer and embedding layer
sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape=[], # shape of inputs coming to our model 
                                        dtype=tf.string, # data type of inputs coming to the USE layer
                                        trainable=False, # keep the pretrained weights (we'll create a feature extractor)
                                        name="USE") 

In [ ]:
# Create model using the Sequential API
model_5 = tf.keras.Sequential([
  sentence_encoder_layer, # take in sentences and then encode them into an embedding
  layers.Dense(64, activation="relu"),
  layers.Dense(1, activation="sigmoid")
], name="model_5_USE")

# Compile model
model_5.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

model_5.summary()

In [ ]:
# Train a classifier on top of pretrained embeddings
model_5_history = model_5.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, 
                                                                     "tf_hub_sentence_encoder")])

In [ ]:
# Make predictions with USE TF Hub model
model_5_pred_probs = model_5.predict(val_sentences)
model_5_pred_probs[:10]

In [ ]:
# Convert prediction probabilities to labels
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))
model_5_preds[:10]

In [ ]:
mod5_prediction=model_5.predict(test_sentences)
mod5_prediction
model_5_preds_final = tf.squeeze(tf.round(mod5_prediction))
model_5_preds_final.shape

In [ ]:
a = tf.constant(model_5_preds_final)
proto_tensor = tf.make_tensor_proto(a)  # convert `tensor a` to a proto tensor
arr=tf.make_ndarray(proto_tensor) 

pred_arr=arr[0]
pred_arr=int(pred_arr)


In [ ]:
# The Predictions for the Universal Sentence Embedding sub
predictions=pd.DataFrame({'id':test_data.id,'target':pred_arr})
predictions.to_csv('submission.csv',index=False)


In [ ]:
predictions.head()

In [ ]:
from IPython.display import FileLink
FileLink('submission.csv')